<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-файл-с-данными-и-изучите-его.-Путь-к-файлу" data-toc-modified-id="Откройте-файл-с-данными-и-изучите-его.-Путь-к-файлу-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте файл с данными и изучите его. Путь к файлу</a></span></li><li><span><a href="#Разделите-исходные-данные-на-обучающую,-валидационную-и-тестовую-выборки." data-toc-modified-id="Разделите-исходные-данные-на-обучающую,-валидационную-и-тестовую-выборки.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделите исходные данные на обучающую, валидационную и тестовую выборки.</a></span></li><li><span><a href="#Исследуйте-качество-разных-моделей,-меняя-гиперпараметры.-Кратко-напишите-выводы-исследования." data-toc-modified-id="Исследуйте-качество-разных-моделей,-меняя-гиперпараметры.-Кратко-напишите-выводы-исследования.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.</a></span><ul class="toc-item"><li><span><a href="#В-качестве-baseline-решения-обучим-dummy-classifier.-accuracy-других-алгоритмов-будем-сравнивать-с-этим-решением" data-toc-modified-id="В-качестве-baseline-решения-обучим-dummy-classifier.-accuracy-других-алгоритмов-будем-сравнивать-с-этим-решением-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>В качестве baseline решения обучим dummy classifier. accuracy других алгоритмов будем сравнивать с этим решением</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающие-деревья" data-toc-modified-id="Решающие-деревья-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Решающие деревья</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#lgbm-base" data-toc-modified-id="lgbm-base-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>lgbm base</a></span></li><li><span><a href="#LightGBM-search" data-toc-modified-id="LightGBM-search-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>LightGBM search</a></span></li><li><span><a href="#Catboost-base" data-toc-modified-id="Catboost-base-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Catboost base</a></span></li><li><span><a href="#Stacking" data-toc-modified-id="Stacking-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>Stacking</a></span></li></ul></li><li><span><a href="#Вывод-:" data-toc-modified-id="Вывод-:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод :</a></span></li></ul></div>

In [2]:
# !pip3 install fast_ml
# !pip install lightgbm
# !pip install --user --upgrade catboost
# !pip install eli5

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from fast_ml.model_development import train_valid_test_split
from eli5.sklearn import PermutationImportance
import eli5

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from lightgbm import LGBMClassifier
from catboost import Pool, CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

from tqdm.notebook import tqdm
%config InlineBackend.figure_format = 'svg'


ModuleNotFoundError: No module named 'lightgbm'

# Откройте файл с данными и изучите его. Путь к файлу

In [ ]:
try:
    df = pd.read_csv('../datasets/users_behavior.csv')
except:
    df = pd.read_csv('/datasets/user_behavior.csv')

display(df)
rs = 41  # random seed
skf = StratifiedShuffleSplit(n_splits=5, random_state=rs)

In [ ]:
sns.pairplot(df, hue='is_ultra');

# Разделите исходные данные на обучающую, валидационную и тестовую выборки.

In [ ]:
# Competition 

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['is_ultra']), df.is_ultra, test_size=.2, random_state=rs,
stratify=df.is_ultra)

In [ ]:
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

In [ ]:
dummy_model = DummyClassifier(strategy="stratified", random_state=rs) 
col_to_rename = ['id', 'is_ultra']

train = X_train.join(y_train).reset_index(drop=True)
test = X_test.reset_index(drop=True)
test_answers = pd.DataFrame(y_test.reset_index(drop=True)).reset_index()
test_answers.columns = col_to_rename

sample_submission = dummy_model.fit(X_train, y_train).predict(X_test)
sample_submission = pd.DataFrame(sample_submission).reset_index()
sample_submission.columns = col_to_rename

train.to_csv('../datasets/train.csv')
test.to_csv('../datasets/test.csv', index=False)
test_answers.to_csv('../datasets/test_answers.csv', index=False)
sample_submission.to_csv('../datasets/sample_submission.csv', index=False)

sample_submission

In [ ]:
predictions = dummy_model.predict(X_test)

In [ ]:
sample_submission['is_ultra'] = predictions

In [ ]:
cb_test = CatBoostClassifier(random_state=rs, learning_rate=0.01, eval_metric='Accuracy')
cb_predicted = cb_test.fit(X_train, y_train).predict(X_test)

In [ ]:
rf_model = RandomForestClassifier(random_state=rs)
rf_model.fit(X_train, y_train)
rf_predicted = rf_model.predict(X_test)

In [ ]:
sample_submission = pd.read_csv('../datasets/sample_submission.csv')
sample_submission['is_ultra'] = rf_predicted
sample_submission.to_csv('../datasets/submission.csv', index=False)

In [ ]:
sample_submission

In [ ]:
X = df.drop(columns=['is_ultra', 'minutes'])
y = df.is_ultra

# X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(
#     df=df.drop(columns=['minutes']),
#     target=['is_ultra'],
#     train_size=.8,
#     valid_size=.1,
#     test_size=.1
# )


# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y,
#                                                   shuffle=True,
                                                  train_size=0.8,
                                                 random_state=rs
)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,
                                                    y_rem,
                                                    test_size=0.5,
#                                                    shuffle=True,
                                                   random_state=rs
)

In [ ]:
def print_score(model_grid):
    predictions = model_grid.predict(X_valid)
    print(f"{model_grid.best_estimator_}\n")
    print("\n", "-"*60)
    print(classification_report(y_valid, predictions))
    print(f"train accuracy: {model_grid.best_score_}")
    print(f"validation accuracy: {accuracy_score(y_valid, predictions)}")
    print(
        f"test accuracy: {accuracy_score(y_test, model_grid.predict(X_test))}\n")
#         print(f"f1 score: {f1_score(y_valid, predictions)}")

In [ ]:
# Функция для возвращения таблицы с метриками
def get_scores(report_df, model, X_test, y_test, name):

    report = pd.DataFrame(columns={'accuracy'}, data=[0])
#     report['ROC-AUC'] = roc_auc_score(y_test,
#                                       model.predict_proba(X_test)[:, 1])
    report['accuracy'] = accuracy_score(y_test, 
                                       model.predict(X_test))
    report['F1'] = f1_score(y_test, model.predict(X_test))
    report['precision_0'] = precision_score(
        y_test, model.predict(X_test), pos_label=0)
    report['precision_1'] = precision_score(
        y_test, model.predict(X_test), pos_label=1)
    report['recall_0'] = recall_score(
        y_test, model.predict(X_test), pos_label=0)
    report['recall_1'] = recall_score(
        y_test, model.predict(X_test), pos_label=1)

    report.index = [name]
    report_df = report_df.append(report)
    return report_df

# Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

## В качестве baseline решения обучим dummy classifier. accuracy других алгоритмов будем сравнивать с этим решением

In [ ]:
dummy_model = DummyClassifier(strategy="most_frequent", random_state=rs) 
dummy_model.fit(X_train, y_train)

dummy_pred = dummy_model.predict(X_valid)
# accuracy_score(y_valid, dummy_pred)


In [ ]:
df_report = pd.DataFrame(columns={'accuracy'}, data=[0])

df_report['accuracy'] = accuracy_score(y_valid, dummy_pred)
df_report['precision_0'] = precision_score(y_valid, dummy_pred, pos_label=0)
df_report['recall_0'] = recall_score(y_valid, dummy_pred, pos_label=0)
df_report['precision_1'] = precision_score(y_valid, dummy_pred, pos_label=1)
df_report['recall_1'] = recall_score(y_valid, dummy_pred, pos_label=1)

df_report.index = ['baseline']
df_report

## Логистическая регрессия

In [ ]:
lr_model = LogisticRegression(random_state=rs)

lr_params = {
    'C':np.logspace(-3,3,7),
    'penalty': ['l1', 'l2'],
    'class_weight': ['Balanced', None]
}
lr_search = RandomizedSearchCV(
                        lr_model,
                        lr_params,
                        cv=5,
                        verbose=3,
                        scoring='accuracy'
)
lr_search.fit(X_train, y_train)



In [ ]:
print_score(lr_search)

In [ ]:
df_report = get_scores(df_report, lr_search.best_estimator_, X_valid, y_valid, 'logistic_regression')
df_report

## Решающие деревья

In [ ]:
tree_model = DecisionTreeClassifier(random_state=rs)

tree_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(3,15),
    'max_features': ['auto', 'sqrt', 'log2'],
    'class_weight': ['balanced', None]
}

tree_search = GridSearchCV(tree_model, 
                           tree_params, 
                           scoring='accuracy',
                           verbose=3,
                           cv=3)
tree_search.fit(X_train, y_train)



In [ ]:
print_score(tree_search)

In [ ]:
df_report = get_scores(
    df_report, tree_search.best_estimator_, X_valid, y_valid, 'tree')
df_report

## Случайный лес

In [ ]:
rf_model = RandomForestClassifier(random_state=41)

# Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(70, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
rf_params = {  'criterion': ['gini', 'entropy'],
               'class_weight': ['balanced', 'balanced_subsample'],
               'n_estimators': np.arange(400,1001,100), # Параметры нашёл, экономлю время ревьюера
               'max_features': ['sqrt', 'log2', 'auto'],
               'max_depth': np.arange(50, 251, 10),
               'min_samples_split': [5],
               'min_samples_leaf': [4], 
               'bootstrap': bootstrap
}

rf_search = RandomizedSearchCV(rf_model, 
                         rf_params, 
                         scoring='accuracy', 
                         verbose=10,
                         cv=5,
                         n_jobs=-1)

rf_search.fit(X_train, y_train)




In [ ]:
print_score(rf_search)

In [ ]:
accuracy_score(y_test, rf_search.predict(X_test))

In [ ]:
df_report = get_scores(
    df_report, rf_search.best_estimator_, X_valid, y_valid, 'random_forest')
df_report

In [ ]:
rf_search.best_estimator_.fit(pd.concat((X_train, X_valid)), pd.concat((y_train, y_valid)))

In [ ]:
concat_pred = rf_search.best_estimator_.predict(X_test)
accuracy_score(y_test, concat_pred)

In [ ]:
df_report = get_scores(
    df_report, rf_search.best_estimator_, 
    X_test, y_test,
    'random_forest_concat'
)

## lgbm base

In [ ]:
lgbm_base = LGBMClassifier(random_state=rs)

lgbm_base.fit(X_train, y_train)

In [ ]:
df_report = get_scores(
    df_report, lgbm_base, X_valid, y_valid, 'lgbm_base')
df_report

## LightGBM search

In [ ]:
lgbm_model = LGBMClassifier(random_state=rs)

lgbm_params = {
                'num_leaves':[20,40,60,80,100], 
                'min_child_samples':[5,10,15],
                'max_depth':[-1,5,10,20],
                'learning_rate':[0.05,0.1,0.2],
                'reg_alpha':[0,0.01,0.03]
    
}

lgbm_search = RandomizedSearchCV(  lgbm_model, 
                                   lgbm_params, 
                                   scoring='accuracy', 
                                   cv=5, 
                                   verbose=3,
                                   n_jobs=-1,
                                   random_state=rs)

lgbm_search.fit(X_train, y_train)



In [ ]:
print_score(lgbm_search)

In [ ]:
df_report = get_scores(
    df_report, lgbm_search.best_estimator_, X_valid, y_valid, 'lgbm_search')
df_report

## Catboost base

In [ ]:
cb_base = CatBoostClassifier(
    random_state=rs,
    eval_metric='Accuracy'
)
cb_base.fit(
    X_train,
    y_train,
    plot=True,
    verbose=False,
    eval_set=(X_valid, y_valid)
)


In [ ]:
df_report = get_scores(df_report, cb_base, X_valid, y_valid, 'catboost_base')

## Stacking

In [ ]:
def compute_meta_feature(model, X_train, X_valid,  y_train, cv, X_test=None,):
    try:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
        model.fit(X_train, y_train)
        if len(X_test) >= 0: 
            return train_answers, model.predict_proba(X_valid)[:, 1], model.predict_proba(X_test)[:, 1]
        else: 
            return train_answers, model.predict_proba(X_valid)[:, 1]
    
    except:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict')#[:, 1]
        model.fit(X_train, y_train)
        return 'except', train_answers, model.predict(X_valid)#[:, 1]

In [ ]:
models = []
models.append(KNeighborsClassifier(n_jobs=-1, n_neighbors=30))

models.append(RandomForestClassifier(max_depth=110, 
                                     min_samples_leaf=4, 
                                     min_samples_split=5,
                                     n_estimators=1400, 
                                     random_state=rs,
                                     n_jobs=-1))
# models.append(RandomForestClassifier(max_depth=7, 
#                                      n_estimators=50, 
#                                      n_jobs=-1, 
#                                      random_state=rs))
models.append(CatBoostClassifier(depth=4,
                                 l2_leaf_reg = 9, 
                                 learning_rate = 0.060000000000000005,
                                verbose=False,
                                eval_metric='Accuracy',
                                random_state=rs))
models.append(LGBMClassifier(
    learning_rate=0.05, 
    min_child_samples=15, 
    num_leaves=20,
    random_state=rs, 
    reg_alpha=0.03,
    n_jobs=-1))


In [ ]:
meta_features_train = np.zeros((X_train.shape[0], 0))
meta_features_valid = np.zeros((X_valid.shape[0], 0))
meta_features_test = np.zeros((X_test.shape[0], 0))

In [ ]:
for model in tqdm(models):
    train, valid, test = compute_meta_feature(model, X_train, X_valid, y_train, 5, X_test)

    meta_features_train = np.append(meta_features_train, train.reshape((train.size, 1)), axis=1)
    meta_features_valid = np.append(meta_features_valid, valid.reshape((valid.size, 1)), axis=1)
    meta_features_test = np.append(meta_features_test, test.reshape((test.size, 1)), axis=1)

In [ ]:
mft = pd.DataFrame(meta_features_train)
mft['mean_pred'] = mft.mean(axis=1)

mft

In [ ]:
stacking_model = LogisticRegression(random_state=rs)
stacking_model.fit(meta_features_train, y_train)

y_train_predicted = stacking_model.predict(meta_features_train)

y_valid_predicted = stacking_model.predict(meta_features_valid)

y_test_predicted = stacking_model.predict(meta_features_test)

In [ ]:
print(
    f"stacking train accuracy: {accuracy_score(y_train, y_train_predicted)}")
print(
    f"stacking validation accuracy: {accuracy_score(y_valid, y_valid_predicted)}")
print(
    f"stacking test accuracy: {accuracy_score(y_test, y_test_predicted)}")

In [ ]:
df_report = get_scores(
    df_report, stacking_model, meta_features_test, y_test, 'stacking').sort_values(by='accuracy', ascending=False)
df_report

In [1]:
cb_base.fit(pd.concat((X_train, X_valid)), pd.concat((y_train, y_valid)))

NameError: name 'cb_base' is not defined

In [ ]:


px.bar(df_report,
       y='accuracy',
       color=df_report.index, 
       log_y=True,
#        range_y=(df_report['accuracy'].min() - .02, df_report['accuracy'].max()  + .02),
       title='ML algoritm comarison'
      )

In [ ]:
perm = (PermutationImportance(rf_search.best_estimator_, scoring='accuracy')
        .fit(X_train, y_train))
eli5.show_weights(perm, feature_names=X_valid.columns.tolist())

# Вывод :



* Данных маловато для полноценного обучения моделей.
* Все алгоритмы из проекта можно назвать разумными, т.к. они работают ощутимо качественнее baseline решения. 
* Три безусловных лидера по accuracy - random_forest, lightGBM, stacking. Последний по метрикам очень близок к lightgbm.
* Странно, что catboost отработал хуже деревьев, скорее всего можно подобрать гиперпараметры лучше для этого алгоритма.
* accuracy - спорная метрика, сильно зависящая от пропорций классов. 


In [ ]:
X_concat = pd.concat((X_train, X_valid))
y_concat = pd.concat((y_train, y_valid))

In [ ]:
# model = CatBoostClassifier(learning_rate=0.03)
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve, select_threshold

# model.fit(train_data, train_labels, verbose=False)
roc_curve_values = get_roc_curve(cb_base, Pool(X_concat, y_concat))

boundary = select_threshold(lgbm_base, 
                            curve=roc_curve_values,  
                            FPR=0.01)
print(boundary)

y_pred = (lgbm_base.fit(X_concat, y_concat).predict_proba(X_test)[:,1] >= boundary).astype(int)
accuracy_score(y_test, y_pred)